<a href="https://colab.research.google.com/github/cazathoth/tfm/blob/main/tfm_jvasquez_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#descarga librerias.
install.packages("tidyverse")
install.packages("arrow")
install.packages("lubridate")
#install.packages('devtools')
#devtools::install_github('catboost/catboost', subdir = 'catboost/R-package')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘assertthat’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
#cargo librerias a utilizar.
library("tidyverse")
library("arrow")
library("lubridate")
#library("catboost")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘arrow’


The following object is masked from ‘package:lubridate’:

    duration


The following object is masked from ‘package:utils’:

    timestamp




In [130]:
#creo directorio y descargo dataset con datos de urgencias respiratorias
dir.create("descargas")
url <- "https://datos.gob.cl/dataset/606ef5bb-11d1-475b-b69f-b980da5757f4/resource/ae6c9887-106d-4e98-8875-40bf2b836041/download/at_urg_respiratorio_semanal.parquet"
nombre_temporal <- "descargas/at_urg_respiratorio_semanal.parquet"
download.file(url, nombre_temporal)

Warning message in dir.create("descargas"):
“'descargas' already exists”


In [131]:
#Descargo datos meteorologicos- temperatura
url <- "https://raw.githubusercontent.com/cazathoth/data/9ed2ff302d64b81e79eab693e77d12f805ed65ba/MERGE_XXXX_DiarioTs_.csv"
nombre_temporal <- "descargas/MERGE_XXXX_DiarioTs_.csv"
download.file(url, nombre_temporal)

In [132]:
#Descargo datos meteorologicos- precipitaciones
url <- "https://raw.githubusercontent.com/cazathoth/data/43bad08be793c38c993d524d182dcb2597e14714/MERGE_DiarioRR.csv"
nombre_temporal <- "descargas/MERGE_DiarioRR.csv"
download.file(url, nombre_temporal)

In [133]:
#Descargo zip datos meteorologicos- humedad
url <- "https://github.com/cazathoth/data/raw/69c13d02f49cd04ee16ab8063f872e2aa9b2632a/MERGE_Humedad.zip"
nombre_temporal <- "descargas/MERGE_Humedad.zip"
download.file(url, nombre_temporal)
#Descomprimo archivo zip
unzip("/content/descargas/MERGE_Humedad.zip", exdir = "/content/descargas/")

In [134]:
#Descargo datos material particulado
url <- "https://raw.githubusercontent.com/cazathoth/data/43bad08be793c38c993d524d182dcb2597e14714/MERGE_MP.csv"
nombre_temporal <- "descargas/MERGE_MP.csv"
download.file(url, nombre_temporal)

In [190]:
# Creo data frame urgencias y filtro por comuna de Santiago, La Serena y Concepción, luego cargo los datos
df_urgr_esp_semanal <- read_parquet("descargas/at_urg_respiratorio_semanal.parquet", col_select = c ('EstablecimientoCodigo','ComunaCodigo', 'ComunaGlosa', 'Anio', 'SemanaEstadistica','OrdenCausa','Causa','NumTotal'))
df_urgr_esp_semanal <- df_urgr_esp_semanal %>% filter (ComunaCodigo %in% c(4101, 8101, 13101) & OrdenCausa ==3)

In [191]:
#ano_base <- "2014-01-01"
#df_datos_temperatura <- read_delim("descargas/MERGE_XXXX_DiarioTs_.csv", delim=";",col_select = c('CodigoNacional','momento','MediaCli_Valor','Maxima_Valor','Minima_Valor'), col_types = cols(momento = col_date(format = "%d-%m-%Y %H:%M:%S")))
#df_datos_temperatura <- df_datos_temperatura %>% filter (momento > ano_base)
#head(df_datos_temperatura,15)

In [192]:
#Cargo data frame temperatura y le cargo datos
#df_datos_temperatura <- read.csv("descargas/MERGE_XXXX_DiarioTs_.csv", header= TRUE, sep=";")
df_datos_temperatura <- read_delim("descargas/MERGE_XXXX_DiarioTs_.csv", delim=";",col_select = c('CodigoNacional','momento','MediaCli_Valor','Maxima_Valor','Minima_Valor'))
head(df_datos_temperatura,5)

Rows: 74539 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (1): momento
dbl (4): CodigoNacional, MediaCli_Valor, Maxima_Valor, Minima_Valor

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


CodigoNacional,momento,MediaCli_Valor,Maxima_Valor,Minima_Valor
<dbl>,<chr>,<dbl>,<dbl>,<dbl>
290004,01-01-1954 12:00:00,16.5,20.6,12.3
290004,02-01-1954 12:00:00,17.1,20.4,13.9
290004,03-01-1954 12:00:00,15.1,20.0,10.2
290004,04-01-1954 12:00:00,16.9,20.8,13.0
290004,05-01-1954 12:00:00,17.7,20.4,15.0


In [193]:
#Cargo data frame humedad y le cargo datos
df_datos_Humedad <- read.csv("descargas/MERGE_Humedad.csv", header= TRUE, sep=";")
head(df_datos_Humedad,5)

,CodigoNacional,momento,HR_Valor
,<int>,<chr>,<dbl>
1,290004,01-01-1954 10:00:00,NA
2,290004,01-01-1954 11:00:00,NA
3,290004,01-01-1954 12:00:00,NA
4,290004,01-01-1954 13:00:00,NA
5,290004,01-01-1954 14:00:00,NA


In [194]:
#Cargo data frame precipitaciones y le cargo datos
#df_datos_Precip <- read.csv("descargas/MERGE_DiarioRR.csv", header= TRUE, sep=";")
df_datos_Precip <- read_delim("descargas/MERGE_DiarioRR.csv", delim=";",col_select = c('CodigoNacional','momento','Total_Valor'))
head(df_datos_Precip,5)

Rows: 66161 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (1): momento
dbl (2): CodigoNacional, Total_Valor

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


CodigoNacional,momento,Total_Valor
<dbl>,<chr>,<dbl>
290004,01-01-1954 12:00:00,0
290004,02-01-1954 12:00:00,0
290004,03-01-1954 12:00:00,0
290004,04-01-1954 12:00:00,0
290004,05-01-1954 12:00:00,0


In [195]:
#Cargo data frame material particulado y le cargo datos
#df_datos_mp25 <- read.csv("descargas/MERGE_MP.csv", header= TRUE, sep=";")
df_datos_mp25 <- read_delim("descargas/MERGE_MP.csv", delim=";",col_select = c('COMUNA', momento = 'FECHA(YYYYMMDD)', registrosValidados='Registros validados'))
head(df_datos_mp25,5)

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 18478 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (1): COMUNA
dbl (1): FECHA(YYYYMMDD)
num (1): Registros validados

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


COMUNA,momento,registrosValidados
<chr>,<dbl>,<dbl>
Concepcion,20090812,NA
Concepcion,20090813,NA
Concepcion,20090814,NA
Concepcion,20090815,NA
Concepcion,20090816,NA


In [196]:
#formateo fechas en material particulado 2.5.
df_datos_mp25$momento <- format(as.Date(as.character(df_datos_mp25$momento),format="%Y%m%d"), format="%d-%m-%Y")
#formateo fechas en temperaturas.
df_datos_temperatura$momento<- format(as.Date(df_datos_temperatura$momento, format = "%d-%m-%Y %H:%M:%S"), format = "%d-%m-%Y")
#formateo fechas en precipitaciones
df_datos_Precip$momento<- format(as.Date(df_datos_Precip$momento, format = "%d-%m-%Y %H:%M:%S"), format = "%d-%m-%Y")
#formateo fechas en humedad
df_datos_Humedad$momento<- format(as.Date(df_datos_Humedad$momento, format = "%d-%m-%Y %H:%M:%S"), format = "%d-%m-%Y")

In [197]:
#Limpio filas con NA en todos los dataframe y chequeo contando posibles NA
df_urgr_esp_semanal <- na.omit(df_urgr_esp_semanal)
df_datos_temperatura <- na.omit(df_datos_temperatura)
df_datos_Precip <- na.omit(df_datos_Precip)
df_datos_Humedad <- na.omit(df_datos_Humedad)
df_datos_mp25 <- na.omit(df_datos_mp25)
sum(!complete.cases(df_urgr_esp_semanal))
sum(!complete.cases(df_datos_temperatura))
sum(!complete.cases(df_datos_Precip))
sum(!complete.cases(df_datos_Humedad))
sum(!complete.cases(df_datos_mp25))


[1] 0

[1] 0

[1] 0

[1] 0

[1] 0

In [198]:
#obtengo n° de semana de las fechas de mediciones (momento) y agrego columna al data frame temperatura
semana<-week(dmy(df_datos_temperatura$momento))
ano <- year(dmy(df_datos_temperatura$momento))
difTemp <- df_datos_temperatura$Maxima_Valor-df_datos_temperatura$Minima_Valor
df_datos_temperatura <- cbind(df_datos_temperatura, NuSemana = semana)
df_datos_temperatura <- cbind(df_datos_temperatura, Anio = ano)
df_datos_temperatura <- cbind(df_datos_temperatura, difTemp = difTemp)
head(df_datos_temperatura,3)

,CodigoNacional,momento,MediaCli_Valor,Maxima_Valor,Minima_Valor,NuSemana,Anio,difTemp
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,290004,01-01-1954,16.5,20.6,12.3,1,1954,8.3
2,290004,02-01-1954,17.1,20.4,13.9,1,1954,6.5
3,290004,03-01-1954,15.1,20.0,10.2,1,1954,9.8


In [199]:
# Obtengo la media y mediana, de la temperatura para cada semana en cada año.
df_datos_temperatura_semanal <- df_datos_temperatura %>% group_by (CodigoNacional,Anio, NuSemana) %>% summarise(medianaClim = median(MediaCli_Valor, na.rm = TRUE),  difMaxMin = median(difTemp, na.rm = TRUE))
#Convierto agrupación con media y mediana a data frame
df_datos_temperatura_semanal <- as.data.frame (df_datos_temperatura_semanal)
#filtro por todo aquellos años >= a 2014
df_datos_temperatura_semanal <- df_datos_temperatura_semanal %>% filter(Anio > 2013 & Anio < 2024)
head(df_datos_temperatura_semanal,5)



`summarise()` has grouped output by 'CodigoNacional', 'Anio'. You can override
using the `.groups` argument.


,CodigoNacional,Anio,NuSemana,medianaClim,difMaxMin
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,290004,2014,1,17.5,7.1
2,290004,2014,2,17.2,6.9
3,290004,2014,3,19.6,8.1
4,290004,2014,4,18.4,6.9
5,290004,2014,5,18.2,7.9


In [200]:
#obtengo n° de semana de las fechas de mediciones (momento) y agrego columna al data frame humedad
semana<-week(dmy(df_datos_Humedad$momento))
ano <- year(dmy(df_datos_Humedad$momento))
df_datos_Humedad <- cbind(df_datos_Humedad, NuSemana = semana)
df_datos_Humedad <- cbind(df_datos_Humedad, Anio = ano)
head(df_datos_Humedad,3)

,CodigoNacional,momento,HR_Valor,NuSemana,Anio
,<int>,<chr>,<dbl>,<dbl>,<dbl>
5113,290004,06-01-1963,95,1,1963
5114,290004,06-01-1963,93,1,1963
5115,290004,06-01-1963,89,1,1963


In [201]:
# Obtengo la media y mediana, de la temperatura para cada semana en cada año.
df_datos_Humedad_semanal <- df_datos_Humedad %>% group_by (CodigoNacional,Anio, NuSemana) %>% summarise(Mediana_HR_Valor = median(HR_Valor, na.rm = TRUE))
#Convierto agrupación con media y mediana a data frame
df_datos_Humedad_semanal <- as.data.frame (df_datos_Humedad_semanal)
#filtro por todo aquellos años >= a 2014
df_datos_Humedad_semanal <- df_datos_Humedad_semanal %>% filter(Anio > 2013 & Anio < 2024)
head(df_datos_Humedad_semanal,5)

`summarise()` has grouped output by 'CodigoNacional', 'Anio'. You can override
using the `.groups` argument.


,CodigoNacional,Anio,NuSemana,Mediana_HR_Valor
,<int>,<dbl>,<dbl>,<dbl>
1,290004,2014,1,80
2,290004,2014,2,81
3,290004,2014,3,68
4,290004,2014,4,75
5,290004,2014,5,77


In [202]:
#obtengo n° de semana de las fechas de mediciones (momento) y agrego columna al data frame precipitaciones
semana<-week(dmy(df_datos_Precip$momento))
ano <- year(dmy(df_datos_Precip$momento))
df_datos_Precip <- cbind(df_datos_Precip, NuSemana = semana)
df_datos_Precip <- cbind(df_datos_Precip, Anio = ano)
head(df_datos_Precip,3)

,CodigoNacional,momento,Total_Valor,NuSemana,Anio
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,290004,01-01-1954,0,1,1954
2,290004,02-01-1954,0,1,1954
3,290004,03-01-1954,0,1,1954


In [203]:
# Agrupo y obtengo la suma de precipitaciones para cada semana en cada año.
df_datos_Precip_semanal <- df_datos_Precip %>% group_by (CodigoNacional,Anio, NuSemana) %>% summarise(Total_Valor_sem = sum(Total_Valor, na.rm = TRUE))
#Convierto agrupación con media y mediana a data frame
df_datos_Precip_semanal <- as.data.frame (df_datos_Precip_semanal)
#filtro por todo aquellos años >= a 2014
df_datos_Precip_semanal <- df_datos_Precip_semanal %>% filter(Anio > 2013 & Anio < 2024)
head(df_datos_Precip_semanal,5)

`summarise()` has grouped output by 'CodigoNacional', 'Anio'. You can override
using the `.groups` argument.


,CodigoNacional,Anio,NuSemana,Total_Valor_sem
,<dbl>,<dbl>,<dbl>,<dbl>
1,290004,2014,1,0
2,290004,2014,2,0
3,290004,2014,3,0
4,290004,2014,4,0
5,290004,2014,5,0


In [204]:
#obtengo n° de semana de las fechas de mediciones (momento) y agrego columna al data frame mp25
semana<-week(dmy(df_datos_mp25$momento))
ano <- year(dmy(df_datos_mp25$momento))
df_datos_mp25 <- cbind(df_datos_mp25, NuSemana = semana)
df_datos_mp25 <- cbind(df_datos_mp25, Anio = ano)
head(df_datos_mp25,3)

,COMUNA,momento,registrosValidados,NuSemana,Anio
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,Concepcion,11-06-2017,33,24,2017
2,Concepcion,12-06-2017,37,24,2017
3,Concepcion,13-06-2017,44,24,2017


In [205]:
# Agrupo y obtengo la suma de precipitaciones para cada semana en cada año.
df_datos_mp25_semanal <- df_datos_mp25 %>% group_by (COMUNA,Anio, NuSemana) %>% summarise(Total_Valor_sem = sum(registrosValidados, na.rm = TRUE))
#Convierto agrupación con media y mediana a data frame
df_datos_mp25_semanal <- as.data.frame (df_datos_mp25_semanal)
#filtro por todo aquellos años >= a 2014
df_datos_mp25_semanal <- df_datos_mp25_semanal %>% filter(Anio > 2013 & Anio < 2024)
head(df_datos_mp25_semanal,5)

`summarise()` has grouped output by 'COMUNA', 'Anio'. You can override using
the `.groups` argument.


,COMUNA,Anio,NuSemana,Total_Valor_sem
,<chr>,<dbl>,<dbl>,<dbl>
1,Concepcion,2017,24,192
2,Concepcion,2017,25,156
3,Concepcion,2017,26,124
4,Concepcion,2017,27,151
5,Concepcion,2017,28,152


In [206]:
count(df_urgr_esp_semanal)
count(df_datos_temperatura_semanal)
count(df_datos_Precip_semanal)
count(df_datos_Humedad_semanal)
count(df_datos_mp25_semanal)

n
<int>
9854


n
<int>
1590


n
<int>
1590


n
<int>
1590


n
<int>
1382


In [207]:
#Aqui estan todas las causas consideradas. La #3 es un total.
df_urgr_esp_semanal %>% select(OrdenCausa, Causa)%>%distinct()%>%arrange(OrdenCausa)

OrdenCausa,Causa
<int>,<chr>
3,TOTAL CAUSAS SISTEMA RESPIRATORIO


In [221]:
df_urgr_esp_semanal %>% group_by (ComunaCodigo, Anio, SemanaEstadistica) %>% summarise(NumTotal = sum(NumTotal, na.rm = TRUE)) %>%filter(Anio > 2013 & Anio < 2024)
#%>%filter (Anio ==2023 & ComunaCodigo ==4101)

`summarise()` has grouped output by 'ComunaCodigo', 'Anio'. You can override
using the `.groups` argument.


ComunaCodigo,Anio,SemanaEstadistica,NumTotal
<chr>,<int>,<int>,<int>
13101,2014,1,295
13101,2014,2,648
13101,2014,3,585
13101,2014,4,500
13101,2014,5,444
13101,2014,6,497
13101,2014,7,379
13101,2014,8,424
13101,2014,9,449


In [217]:
#Filtro el dataset par las regiones 8,4 y 13, tambien la causa 3 que es solo totalizador de urgencias reportadas
#df_urgr_esp_semanal<- df_urgr_esp_semanal %>% filter (RegionCodigo %in% c(8,4,13) & OrdenCausa != 3)
#count(df_urgr_esp_semanal)
#head(df_urgr_esp_semanal,5)
df_urgr_esp_semanal %>% filter (ComunaGlosa == 'La Serena' & Anio == 2023 & SemanaEstadistica == 47) %>%arrange (Anio)
#%>% select(ComunaCodigo, ComunaGlosa) %>% filter(ComunaGlosa == 'Santiago')

EstablecimientoCodigo,ComunaCodigo,ComunaGlosa,Anio,SemanaEstadistica,OrdenCausa,Causa,NumTotal
<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<int>
105100,4101,La Serena,2023,47,3,TOTAL CAUSAS SISTEMA RESPIRATORIO,247
105819,4101,La Serena,2023,47,3,TOTAL CAUSAS SISTEMA RESPIRATORIO,276
200105,4101,La Serena,2023,47,3,TOTAL CAUSAS SISTEMA RESPIRATORIO,85
105825,4101,La Serena,2023,47,3,TOTAL CAUSAS SISTEMA RESPIRATORIO,153
105826,4101,La Serena,2023,47,3,TOTAL CAUSAS SISTEMA RESPIRATORIO,237
105802,4101,La Serena,2023,47,3,TOTAL CAUSAS SISTEMA RESPIRATORIO,161


In [209]:
#Obtengo el total de urgencias reportadas en todos los años por las 3 regiones.
#df_urgr_esp_semanal%>% group_by(RegionGlosa) %>% summarise(Total = sum(NumTotal, na.rm = TRUE))

In [210]:
# Grafico la cantidad de casos por región
#df_plot1<-df_urgr_esp_semanal%>% group_by(RegionGlosa) %>% summarise(Total = sum(NumTotal, na.rm = TRUE))
#options(repr.plot.width = 10, repr.plot.height = 10)
#ggplot(df_plot1, aes(x=RegionGlosa, y= Total, fill= RegionGlosa)) + geom_bar(stat="identity") +
#scale_y_continuous(labels = scales::comma_format(accuracy = 1))+
#theme(axis.text.x = element_text(angle = 75, vjust = .5))+
#labs(x = "Región", y = "Total Urgencias", title = "Total Urgencias x Región")

In [211]:
# Obtengo cantidad de urgencias por año y region
#df_resumen <- df_urgr_esp_semanal %>%
#  group_by(Anio, RegionGlosa) %>%
#  summarise(Total = sum(NumTotal, na.rm = TRUE))
#  df_resumen

In [212]:
# Obtengo cantidad de urgencias por año, semana y region.
#df_resumen <- df_urgr_esp_semanal %>%
#  group_by(Anio, SemanaEstadistica, Causa) %>%
#  summarise(Total = sum(NumTotal, na.rm = TRUE))
#  df_resumen

In [213]:
#df_agrupado <- df_urgr_esp_semanal %>%
 # group_by(Anio) %>%
  #summarise(Total = sum(NumTotal, na.rm = TRUE))
#options(repr.plot.width = 15, repr.plot.height = 15)
#ggplot(df_agrupado, aes(x = factor(Anio), y = Total)) +
 # geom_bar(stat = "identity", fill = "steelblue") +
 # labs(x = "Año", y = "Total Urgencias", title = "Urgencias por Año") +
 # theme_minimal()